# EEG signal denosing using Wavelet transform



> Analyzing Electroencephalogram (EEG) signal is a challenge due to the various artifacts used by Electromyogram, eye blink and Electrooculogram. The present de-noising techniques that are based on the frequency selective filtering suffers from a substantial loss of the EEG data. Noise removal using wavelet has the characteristic of preserving signal uniqueness even if noise is going to be minimized. To remove noise from EEG signal, this research employed discrete wavelet transform. Root mean square difference has been used to find the usefulness of the noise elimination. In this research, four different discrete wavelet functions have been used to remove noise from the Electroencephalogram signal gotten from two different types of patients (healthy and epileptic) to show the effectiveness of DWT on EEG noise removal. The result shows that the WF orthogonal meyer is the best one for noise elimination from the EEG signal of epileptic subjects and the WF Daubechies 8 (db8) is the best one for noise elimination from the EEG signal on healthy subjects. 

Mamun, M., Al-Kadi, M., & Marufuzzaman, M. (2013). Effectiveness of Wavelet Denoising on Electroencephalogram Signals. Journal of Applied Research and Technology, 11(1). https://doi.org/10.1016/S1665-6423(13)71524-4

For more details, pleas visit [Effectiveness of Wavelet Denoising on Electroencephalogram Signals](https://www.sciencedirect.com/science/article/pii/S1665642313715244)

### Please Upvote if you Find this Useful :)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pywt

In [ ]:
# from this notebook: "Harmful Brain Activity Plotting eeg/spec(https://www.kaggle.com/code/ybbbat/harmful-brain-activity-plotting-eeg-spec)"
def plot_eeg(df, moving_avg=1):
    fig, axs = plt.subplots(20, 1, figsize=(30, 15), sharex=True)
    for i, ax in enumerate(axs):
        ax.plot(df.iloc[:,i], color="black")
        ax.set_ylabel(df.columns[i], rotation=0)
        ax.set_yticklabels([])
        ax.set_yticks([])
        ax.set_xticks([])
        ax.spines[["top", "bottom", "left", "right"]].set_visible(False)

# Denoising function

In [ ]:
def maddest(d, axis=None):
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)

def denoise(x, wavelet='haar', level=1):
    ret = {key:[] for key in x.columns}
    
    for pos in x.columns:
        coeff = pywt.wavedec(x[pos], wavelet, mode="per")
        sigma = (1/0.6745) * maddest(coeff[-level])

        uthresh = sigma * np.sqrt(2*np.log(len(x)))
        coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:])

        ret[pos]=pywt.waverec(coeff, wavelet, mode='per')
    
    return pd.DataFrame(ret)

# List of mother Function you can use for wavelet transform

In [ ]:
print("The mother function you can use")
print(pywt.wavelist())

In [ ]:
# get eeg signal
df = pd.read_csv("/kaggle/input/hms-harmful-brain-activity-classification/train.csv")
row = df.sample(n=1, random_state=60).iloc[0,]

# Befor Denoising

In [ ]:
eeg = pd.read_parquet("/kaggle/input/hms-harmful-brain-activity-classification/train_eegs/"+str(row["eeg_id"])+".parquet")
eeg = eeg[int(row["eeg_label_offset_seconds"])*200:int(row["eeg_label_offset_seconds"])*200+50*200]
plot_eeg(eeg)
plt.savefig("bef.png")

# After Denoising

In [ ]:
eeg_denoised = denoise(eeg, wavelet="db8")
plot_eeg(eeg_denoised)
plt.savefig("aft.png")